2025/11/4
mlflowをngrokのトンネリングを用いてgoogle colabから接続する方法
方法策定はCursorにて実行。
cursorのAgentタイトル：Google Colabでngrok経由のmlflow使用

In [1]:
!pip -q install mlflow==2.16.2 pyngrok==7.2.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 14.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 18.8 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.7/774.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.8 MB/s eta 0:00:00


In [2]:
from pyngrok import ngrok
NGROK_AUTHTOKEN = "1wvkoOmDIsa51PzNifD0Ho1XtCy_87DHFX66mnoB1ZKJSuqXG"  #
ngrok.set_auth_token(NGROK_AUTHTOKEN)

In [3]:
import subprocess, os, time, signal, psutil

# 既存の5000番プロセスがあれば掃除
for p in psutil.process_iter(attrs=["pid","name","cmdline"]):
    try:
        if p.info["cmdline"] and "mlflow" in " ".join(p.info["cmdline"]):
            p.kill()
    except Exception:
        pass

os.makedirs("/content/mlruns", exist_ok=True)
mlflow_cmd = [
    "mlflow", "server",
    "--host", "0.0.0.0",
    "--port", "5000",
    "--backend-store-uri", "sqlite:////content/mlflow.db",
    "--default-artifact-root", "file:///content/mlruns"
]
mlflow_proc = subprocess.Popen(mlflow_cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
time.sleep(3)
print("MLflow server started (port 5000)")

MLflow server started (port 5000)


In [4]:
public_tunnel = ngrok.connect(5000)  # http トンネル
public_url = public_tunnel.public_url
public_url

'https://6f9f235c99bf.ngrok-free.app'

In [5]:
import mlflow

mlflow.set_tracking_uri(public_url)  # ngrok経由のURLを使用
mlflow.set_experiment("colab-demo")

with mlflow.start_run(run_name="hello-ngrok"):
    mlflow.log_param("alpha", 0.1)
    mlflow.log_metric("rmse", 0.987)

2025/12/31 06:34:42 INFO mlflow.tracking.fluent: Experiment with name 'colab-demo' does not exist. Creating a new experiment.
2025/12/31 06:34:45 INFO mlflow.tracking._tracking_service.client: 🏃 View run hello-ngrok at: https://6f9f235c99bf.ngrok-free.app/#/experiments/1/runs/a5e10003c22d4bb0b27f8fc6fd7be6c5.
2025/12/31 06:34:45 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://6f9f235c99bf.ngrok-free.app/#/experiments/1.
